In [4]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary

# Données d'exemple (à remplacer par vos données réelles)
equipes = ["A", "B", "C", "D"]
semaines = range(1, 5)  # Exemple : 4 semaines
emissions_avion = {("A","B"): 100, ("A","C"): 150, ("A","D"): 200,
                   ("B","A"): 100, ("B","C"): 50, ("B","D"): 120,
                   ("C","A"): 150, ("C","B"): 50, ("C","D"): 80,
                   ("D","A"): 200, ("D","B"): 120, ("D","C"): 80}
emissions_train = {("A","B"): 50, ("A","C"): 70, ("A","D"): 90,
                   ("B","A"): 50, ("B","C"): 20, ("B","D"): 40,
                   ("C","A"): 70, ("C","B"): 20, ("C","D"): 30,
                   ("D","A"): 90, ("D","B"): 40, ("D","C"): 30}

# Initialisation du problème
prob = LpProblem("Calendrier_Ligue1_Emissions", LpMinimize)

# Variables de décision
x = LpVariable.dicts("Match", [(i,j,w) for i in equipes for j in equipes if i!=j for w in semaines], cat=LpBinary)
y_avion = LpVariable.dicts("Avion", [(i,j,w) for i in equipes for j in equipes if i!=j for w in semaines], cat=LpBinary)
y_train = LpVariable.dicts("Train", [(i,j,w) for i in equipes for j in equipes if i!=j for w in semaines], cat=LpBinary)

# Fonction objectif : minimiser les émissions
prob += lpSum(y_avion[(i,j,w)] * emissions_avion[(j,i)] + y_train[(i,j,w)] * emissions_train[(j,i)]
              for i in equipes for j in equipes if i!=j for w in semaines)

# Contraintes
for i in equipes:
    for w in semaines:
        prob += lpSum(x[(i,j,w)] for j in equipes if j!=i) + lpSum(x[(j,i,w)] for j in equipes if j!=i) == 1

for i in equipes:
    for j in equipes:
        if i != j:
            prob += lpSum(x[(i,j,w)] for w in semaines) == 1

for i in equipes:
    for j in equipes:
        if i != j:
            for w in semaines:
                prob += y_avion[(i,j,w)] + y_train[(i,j,w)] == x[(i,j,w)]

# Résolution
prob.solve()

# Affichage des résultats
print("Statut :", prob.status)
print("Émissions totales minimisées :", prob.objective.value())


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/50abedc863514ca2bc4da1ba35d87815-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/50abedc863514ca2bc4da1ba35d87815-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 81 COLUMNS
At line 754 RHS
At line 831 BOUNDS
At line 976 ENDATA
Problem MODEL has 76 rows, 144 columns and 288 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Statut : -1
Émissions totales minimisées : 560.0


In [2]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 46.6 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.
